# **Elaborazione di Immagini Mediche**
## Vessel Wall Segmentation Challenge - A.A. 2021/22 

### Script di Training

Rigazio Sofia, Roccaro Lucia, Romano Anastasio, Ruzzante Elena

Collegamento a Google Drive e installazione delle librerie necessarie

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install tensorflow==2.1.0
!pip install keras==2.3.1
!pip install segmentation_models==1.0.1
!pip install h5py==2.10.0



Importazione delle librerie necessarie



In [ ]:
import os
import numpy as np
import pickle

from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

from segmentation_models import Unet

Definizione dei path

In [ ]:
current_dir = '/content/drive/MyDrive/Colab Notebooks/EIM/VesselWallSegmentationChallenge'

JSON_path = os.path.join(current_dir,'PICKLE_and_JSON')
PICKLE_ICA_path = os.path.join(JSON_path,'ICA')
PICKLE_ECA_path = os.path.join(JSON_path,'ECA')

Definizione dei parametri e della funzione di Data Augmentation (comune alle due reti)

In [ ]:
# Parametri
NUM_CLASSES = 3 # vogliamo segmentare tre classi per ogni rete: lume, wall e background
IMG_CHANNELS = 1 # Le immagini fornite sono grayscale

# Data augmentation (Training set)
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
image_datagen = ImageDataGenerator(
		rotation_range = 15, # per aggiungere variabilità (rotazioni randomiche di 15°)
		fill_mode = 'reflect')

# Data augmentation (Validation set) -> non settiamo parametri perché fare data
# augmentation sul Validation set è un errore
val_datagen = ImageDataGenerator()

# Generator
seed = 1
def XYaugmentGenerator(X1, y, seed, batch_size):
	genX1 = image_datagen.flow(X1, y, batch_size=batch_size, seed=seed)
	genX2 = image_datagen.flow(y, X1, batch_size=batch_size, seed=seed)
	while True:
		X1i = genX1.next()
		X2i = genX2.next()
		yield X1i[0], X2i[0]

## Preallenamento ICA per 5 epoche

Load delle matrici salvate e delle informazioni sulle ROI e sulle immagini da fornire alla rete

In [ ]:
with open(os.path.join(PICKLE_ICA_path,'train_ROI_biforcazioni.pickle'), 'rb') as f:
  X_train, Y_train, _, _, _ = pickle.load(f)
n_images_train = X_train.shape[0]

with open(os.path.join(PICKLE_ICA_path,'validation_ROI_biforcazioni.pickle'), 'rb') as f:
  X_val, Y_val, _, _, _ = pickle.load(f)
n_images_val = X_val.shape[0]

with open(os.path.join(PICKLE_ICA_path,'ROI_NET_informations.pickle'), 'rb') as f:
  _, _, _, _, NET_IMG_HEIGHT, NET_IMG_WIDTH = pickle.load(f)

Definizione del modello U-NET per la rete ICA

In [ ]:
BACKBONE = 'resnet18'
model = Unet(backbone_name = BACKBONE,
            input_shape = (NET_IMG_HEIGHT,NET_IMG_WIDTH,IMG_CHANNELS),
            encoder_weights = None,
            decoder_block_type = 'transpose',
            classes = NUM_CLASSES,
            activation = 'softmax')

# activation: https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/#:~:text=node%20per%20class%2C-,softmax,-activation.

# Definizione algoritmo di ottimizzazione e funzione di loss
model.compile('Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Allenamento della rete ICA per 5 epoche

In [ ]:
# Parametri della rete
batch_size = 64
n_epochs = 5

# Checkpoint definition
csv_logger = CSVLogger('./log.out', append=True, separator=';')
callbacks_list = [csv_logger]

# Train model
results = model.fit_generator(XYaugmentGenerator(X_train,Y_train,seed,batch_size), 
                              steps_per_epoch = np.ceil(float(n_images_train)/float(batch_size)),
                              validation_data = val_datagen.flow(X_val,Y_val,batch_size), 
                              validation_steps = np.ceil(float(n_images_val)/float(batch_size)),
                              shuffle = True,
                              epochs = n_epochs,
                              callbacks = callbacks_list)

Salvataggio del modello ICA allenato per 5 epoche

In [ ]:
model_path = os.path.join(current_dir,'Trained_Nets')
try:
  os.mkdir(model_path)
except:
  pass

net_name = 'ICA_5epochs.h5'
model.save(os.path.join(model_path,net_name))

## ICA

Load del modello ICA allenato per 5 epoche

In [ ]:
net_name = 'ICA_5epochs.h5'
model_path = os.path.join(current_dir, 'Trained_Nets', net_name)
model = load_model(model_path)

# Definizione algoritmo di ottimizzazione e funzione di loss
model.compile('Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Allenamento della rete ICA partendo dai pesi preallenati della rete ICA per 5 epoche

In [ ]:
# Parametri della rete
batch_size = 64
n_epochs = 40

# Checkpoint definition
csv_logger = CSVLogger('./log.out', append=True, separator=';')
earlystopping = EarlyStopping(monitor = 'val_categorical_accuracy', verbose = 1, min_delta = 0.005, patience = 8, mode = 'max')
callbacks_list = [csv_logger, earlystopping]#, checkpoint]

# Train model
results = model.fit_generator(XYaugmentGenerator(X_train,Y_train,seed,batch_size), 
                              steps_per_epoch = np.ceil(float(n_images_train)/float(batch_size)),
                              validation_data = val_datagen.flow(X_val,Y_val,batch_size), 
                              validation_steps = np.ceil(float(n_images_val)/float(batch_size)),
                              shuffle = True,
                              epochs = n_epochs,
                              callbacks = callbacks_list)

Salvataggio del modello ICA allenato

In [ ]:
model_path = os.path.join(current_dir,'Trained_Nets')
try:
  os.mkdir(model_path)
except:
  pass

net_name = 'trained_net_ICA.h5'
model.save(os.path.join(model_path,net_name))

## ECA

Load delle matrici salvate e delle informazioni sulle ROI e sulle immagini da fornire alla rete

In [ ]:
with open(os.path.join(PICKLE_ECA_path,'train_ROI_biforcazioni.pickle'), 'rb') as f:
  X_train, Y_train, _, _, _ = pickle.load(f)
n_images_train = X_train.shape[0]

with open(os.path.join(PICKLE_ECA_path,'validation_ROI_biforcazioni.pickle'), 'rb') as f:
  X_val, Y_val, _, _, _ = pickle.load(f)
n_images_val = X_val.shape[0]

with open(os.path.join(PICKLE_ECA_path,'ROI_NET_informations.pickle'), 'rb') as f:
  _, _, _, _, NET_IMG_HEIGHT, NET_IMG_WIDTH = pickle.load(f)

Load del modello ICA allenato per 5 epoche

In [ ]:
net_name = 'ICA_5epochs.h5'
model_path = os.path.join(current_dir, 'Trained_Nets', net_name)
model = load_model(model_path)

# Definizione algoritmo di ottimizzazione e funzione di loss
model.compile('Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Allenamento della rete ECA partendo dai pesi preallenati della rete ICA per 5 epoche

In [ ]:
# Parametri della rete
batch_size = 16
n_epochs = 40

# Checkpoint definition
csv_logger = CSVLogger('./log.out', append=True, separator=';')
earlystopping = EarlyStopping(monitor = 'val_categorical_accuracy', verbose = 1, min_delta = 0.005, patience = 12, mode = 'max')
callbacks_list = [csv_logger, earlystopping]

# Train model
results = model.fit_generator(XYaugmentGenerator(X_train,Y_train,seed,batch_size), 
                              steps_per_epoch = np.ceil(float(n_images_train)/float(batch_size)),
                              validation_data = val_datagen.flow(X_val,Y_val,batch_size), 
                              validation_steps = np.ceil(float(n_images_val)/float(batch_size)),
                              shuffle = True,
                              epochs = n_epochs,
                              callbacks = callbacks_list)

Salvataggio del modello ECA allenato

In [ ]:
model_path = os.path.join(current_dir,'Trained_Nets')
try:
  os.mkdir(model_path)
except:
  pass

net_name = 'trained_net_ECA.h5'
model.save(os.path.join(model_path,net_name))